<h2><center> درس پردازش زبان‌های طبیعی </center></h2>
<h2><center> علیرضا فرج تبریزی 97102185 </center></h2>
<h2><center> تمرین 1 </center></h2>

<h5 style='direction:rtl;'> در این تمرین به دلیل علاقه شخصی ام به تاریخ سراغ کتاب تاریخ تمدن ویل دورانت رفتم که pdf ترجمه آن از <a href = "https://dl.bookiha.com/falsafi/tarikhe%20tamadon_vil%20dorant(www.bookiha.ir).pdf?_gl=1*r3pug1*_ga*MTc1OTQyNDkyNC4xNzExOTQwNjcx*_ga_P02M25RN1S*MTcxMzkwNzc4Ni4yLjAuMTcxMzkwNzc4Ni4wLjAuMA..">اینجا</a> قابل دانلود است. همچنین فایل txt آن در گیت هابم قرار دارد که در خطوط اول کد دانلود شده و به محلی که خود نوتبوک قرار دارد منتقل می شود. </h5>



<h5 style='direction:rtl;'> چنانچه کد روی کلب اجرا می شود باید تمام دستورات زیر از حالت کامنت خارج و اجرا شود تا کتابخانه های لازم نصب شوند. </h5>

## Necessary Packages and Files

In [1]:
#!pip install hazm
#!pip install dadmatools
#!pip install yake
!git clone https://github.com/AlirezaFT78/ARFATool.git
!git clone https://github.com/AlirezaFT78/NLP_Homeworks.git


fatal: destination path 'ARFATool' already exists and is not an empty directory.
fatal: destination path 'NLP_Homeworks' already exists and is not an empty directory.


## Importing Libraries

In [2]:
import tqdm
import random
import pandas as pd
import numpy as np

## Importing Text

In [3]:
file1 = open('NLP_Homeworks/HW1/Tarikhe_Will_Durant.txt')
Lines = file1.readlines()
file1.close()
line_splited_text = []
for line in Lines:
  line = line.strip()
  line_splited_text.append(line)

<h5 style='direction:rtl;'> در pdf کتاب چند صفحه اول اطلاعات ناشر است که اطلاعات مفیدی برای پردازش محتوای کتاب نیست به همین دلیل خطوطی که قبل از شروع صفحه سوم کتاب هستند حذف می شوند </h5>

In [4]:
# finding the start line of each book from the 11
Book_Beginning = []
for i in range (len(line_splited_text)):
  if line_splited_text[i][0:12]== '\u202bﻓﺼﻞ اول \u202a :':
    Book_Beginning.append(i)

# remove the first unimportant pages
line_splited_text = line_splited_text[Book_Beginning[0]:]

## Removing Page Numbers and Extra Spaces

<h5 style='direction:rtl;'> متن کتاب به صورت خط به خط خوانده شده و هر خط یک عنصر لیست است برای تحلیل محتوایی صحیح باید متن به سطح جمله برود اما بسیاری از مواقع یک جمله در پایان یک صفحه تمام نمی شود و ادامه آن در صفحه بعد است برای رفع این مشکل و رسیدن به متن خالص باید خطوطی که شماره صفحه است پاک شود. </h5>

In [5]:
# making a dictionary for persion to english numbers
number_dict_by_code = {1632+i :i+48 for i in range(10)}
dict2 = {1776+i :i+48 for i in range(10)}
number_dict_by_code.update(dict2)

number_dict_by_char = {chr(1632+i) :chr(i+48) for i in range(10)}
dict2 = {chr(1776+i) :chr(i+48) for i in range(10)}
number_dict_by_char.update(dict2)
print(number_dict_by_char)

{'٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4', '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9', '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4', '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'}


<h5 style='direction:rtl;'> این تابع چک می کند که آیا unicode هر chr از خط دریافتی مربوط به یک عدد فارسی است یا نه و اگر هست آیا با متغیری که شماره صفحه ای که دنبال آن هستیم برابر است یا نه و اگر هست ایندکس آن خط را ذخیره می کند. دلیل وجود متغیر page این است که هر خطی از کتاب که در آن عددی آمده را پاک نکنیم. </h5>

In [6]:
def index_of_pages(txt, first_page, page_miss_tresh, num_dic):
  index = []
  page = first_page
  c = 0
  pages_missed_count = 0
  for i in range(len(txt)):
    num = ''

    if c > page_miss_tresh: #checks if we missed the page number and stop the bug propagation
      pages_missed_count+=1
      page+=1
      c = 0

    for j in txt[i]:
      if (1775<ord(j)<1786) or (1631<ord(j)<1642): #checks if the line include persian number
        num+=j
    c+=1
    try:
      if int(num.translate(num_dic)) == page:
        index.append(i)
        page+=1
        c = 0
    except:
      pass

  return index, pages_missed_count

<h5 style='direction:rtl;'> به کمک حلقه زیر و با کمی سعی و خطا ترشهولد اسکیپ کردن شماره صفحه را روی 70 قرار می دهیم چرا که اولا یک صفحه حداکثر به 70 خط تبدیل شده ثانیا هیچ جا دو صفحه متوالی کمتر 70 خط نیستند و تنها صفحه گمشده به دلیل تبدیل نشدن آن از pdf به متن است. </h5>

In [7]:
# finding the index of page numbers and maximum line per page
page_index, missed_pages = index_of_pages(line_splited_text, 3, 70, number_dict_by_code)
max = 0
k = 0

for i in range(len(page_index)-1):
  if page_index[i+1] - page_index[i] > max:
     num1 = ''
     num2 = ''

     for j in line_splited_text[page_index[i+1]]:
      if (1775<ord(j)<1786) or (1631<ord(j)<1642):
        num1+=j
     for j in line_splited_text[page_index[i]]:
      if (1775<ord(j)<1786) or (1631<ord(j)<1642):
        num2+=j

     if (int(num1.translate(number_dict_by_code))-int(num2.translate(number_dict_by_code)))==1:
      max = page_index[i+1] - page_index[i]
      k = i

print("number of missed page numebrs:", missed_pages)
print("maximum lines in a single page after normalization: ", max)

number of missed page numebrs: 1
maximum lines in a single page after normalization:  70


<h5 style='direction:rtl;'> بعد از خطی که شماره صفحه در آن آمده یک خط خالی هم وجود دارد که هردوی این مورد در حلقه زیر حذف می شوند. </h5>

In [8]:
# removing the lines that are page numbers
for i in range(len(page_index)):
  line_splited_text.pop(page_index[i]-i)

# removing empty space at start of each page
c = 0
for i in range(len(line_splited_text)):
  if line_splited_text[i-c] == '':
    line_splited_text.pop(i-c)
    c+=1

<h5 style='direction:rtl;'> در این بخش برای تحلیل محتوایی متن کتاب آن را بر اساس 11 جلدش تقسیم می کنیم. این تقسیم بندی به کمک پترن دیده شده در کتاب است که هم فهرست اول هر کتاب را حذف می کند هم قسمت مراجع انگلیسی آخر هر کتاب را حذف می کند و بالاخره متن کتاب آمده پردازش می شود. </h5>

In [9]:
# finding the start of each book (to remove the book index pages)
Book_Beginning = []
for i in range (len(line_splited_text)):
  if line_splited_text[i] == '\u202bﻓﺼﻞ اول\u202c':
    Book_Beginning.append(i)

# finding the end of each book (to remove the book reference pages)
Book_Ending = []
for i in range (len(line_splited_text)):
  if line_splited_text[i][-12:] == 'ﻣﺘﻦ اﻧﮕﻠﯿﺴﯽ\u202c':
    Book_Ending.append(i)

#spliting the text by books (Will Durant's 'History of Civilization' is an 11-volume set of books)
Books_List = []
for i in range(len(Book_Beginning)):
    Books_List.append(line_splited_text[Book_Beginning[i]:Book_Ending[i]])


Books_Names = ['مشرق زمین، گهواره تمدن','یونان باستان','سزار و مسیح','عصر ایمان','رنسانس','اصلاح دین',
                            'آغاز عصر خرد','عصر لویی چهاردهم','عصر ولتر','روسو و انقلاب','عصر ناپلئون']

Books_Dict_Line  = {}
for i,j in zip(Books_Names, Books_List):
    Books_Dict_Line [i] = j


<h5 style='direction:rtl;'> در این قسمت زیر اثر سه تابع نرمال کننده مقایسه شده است که دلیل اینکه خودم یک نرمالایزر نوشته ام را در ادامه توضیح می دهم. </h5>

## Normalizers Comparison

In [10]:
import hazm

hazm_normalizer = hazm.Normalizer()
text_normalized_hazm = [hazm_normalizer.normalize(x) for x in tqdm.tqdm(line_splited_text)]

100%|██████████| 257180/257180 [00:58<00:00, 4430.81it/s]


In [11]:
import dadmatools.normalizer as dtn

dadma_normalizer = dtn.Normalizer(
    full_cleaning=True,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

text_normalized_dadma = [dadma_normalizer.normalize(x) for x in tqdm.tqdm(line_splited_text)]

  0%|          | 0/257180 [00:00<?, ?it/s]

100%|██████████| 257180/257180 [02:36<00:00, 1638.30it/s]


<h5 style='direction:rtl;'> متن این کتاب با unicodeای غیر استاندارد نوشته شده است در این مدل از نوشتار فارسی(عربی) هر حرف 4 یا 3 کارکتر جداگانه برای خودش دارد حالت ابتدا، وسط، آخر و تنها البته برخی حروف مانند ر، و، د و... تفاوتی در حالت آخر و وسطشان وجود ندارد.
مزیت این نوشتار به نوشتار عادی این است که نیازی به کاراکتر نیم فاصله ندارد چرا که اگر نیاز به نیم فاصله بعد از یک حرف است کافی است کاراکتر حالت آخر آن را استفاده شود چرا که کاراکتر حالت آخر به حرف بعدی نمی چسبد
این unicode توسط هیچ کدام از کتابخانه های هضم و دادما پشتیبانی درستی نمی شود به همین دلیل من یک نرمالایزر کامل نوشتم که هم این فرم خاص را (Arabic Presentation Form A and B) را ساپورت کند هم برخی پردازش های دیگر را انجام دهد. </h5>

<h5 style='direction:rtl;'> در کلاس نرمالایزی که نوشته ام 4 متغیر ورودی به ترتیب: 1-اعداد فارسی را به انگلیسی تبدیل می کند 2-اعلائم نگارشی به جز نقطه را پاک می کند 3-انواع اعراب و تنویا را پاک میکند 4-هر کاراکتری غیر از الفبا، حروف و اعداد انگلیسی و فارسی را حذف می کند. </h5>

In [12]:
# I developed a normalizer because dadma and hazm do not support my text unicodes
from ARFATool import normalizer as arn

alireza_normalizer = arn.Normalizer(eng_num = True, rem_punc = True, rem_vow = True, standard = True)
text_normalized_alireza = [alireza_normalizer.normalize(x) for x in tqdm.tqdm(line_splited_text)]

  1%|          | 1710/257180 [00:00<00:14, 17092.19it/s]

100%|██████████| 257180/257180 [00:12<00:00, 19898.62it/s]


<h5 style='direction:rtl;'> مثال زیر که یک جمله از کتاب است به خوبی تفاوت فرم های مختلف یونیکد فارسی و اثر نرمالیزر مختلف را نشان می دهد: </h5>

In [23]:
txt1 = 'ﯾﺎدﺗﻮ آﻧﭽﻨﺎن دﯾﻮاﻧﻪام ﮐﺮده اﺳﺖ ﮐﻪ ﺑﻪ اﻧﺪﯾﺸﻪﻫﺎي ﺧﻮﯾﺶ اﻃﻤﯿﻨﺎن ﻧﺪارم' # Arabic Presentation Form A
txt2 = 'یادتو آنچنان دیوانه\u200cام کرده است که به اندیشه\u200cهای خویش اطمینان ندارم' # Arabic
print(hazm_normalizer.normalize(txt1))
print(dadma_normalizer.normalize(txt1))
print(alireza_normalizer.normalize(txt1))
print(txt2)

ﯾادتو آنچنان دﯾوانهام کرده است که به اندﯾشه‌های خوﯾش اطمﯿنان ندارم
یﺎدتو آذچنﺎن دیواذهام اذدیشههﺎی اطمینﺎن ذدارم
یادتو آنچنان دیوانه‌ام کرده است که به اندیشه‌های خویش اطمینان ندارم
یادتو آنچنان دیوانه‌ام کرده است که به اندیشه‌های خویش اطمینان ندارم


<h5 style='direction:rtl;'> عملکرد هضم نسبتا قابل قبول است اما اولا کارکتر وسط برخی حروف را ساپورت نمی کند و از آن مهم تر نیم فاصله بعد کاراکتر آخر حروف را قرار نمی دهد. اگر برایتان سوال است که اگر نرمالیزر من بعد از همه حروف آخر نیم فاصله قرار دهد گاهی نیم فاصله اضافه تولید می شود! باید بگویم که خیر و اگر نیم فاصله و فاصله عادی کنار هم قرار گیرند نیم فاصله حذف می شود.  <h5>

In [36]:
# there are many formations of arabic unicodes that neither hazm nor dadmatools unify to standard form
random_line = random.sample(range(0,len(line_splited_text)),10)

df = pd.DataFrame({'original_text':[line_splited_text[idx] for idx in random_line],
                   'hazm_normalized':[text_normalized_hazm[idx] for idx in random_line],
                   'dadma_normalized':[text_normalized_dadma[idx] for idx in random_line],
                   'alireza_normalized':[text_normalized_alireza[idx] for idx in random_line]
                  })

pd.set_option('display.max_colwidth', 100)
display(df)

,original_text,hazm_normalized,dadma_normalized,alireza_normalized
0,‫ﯾﺎدﺗﻮ آﻧﭽﻨﺎن دﯾﻮاﻧﻪام ﮐﺮده اﺳﺖ ﮐﻪ ﺑﻪ اﻧﺪﯾﺸﻪﻫﺎي ﺧﻮﯾﺶ اﻃﻤﯿﻨﺎن ﻧﺪارم‪ .‬ﯾﮏ ﭼﯿﺰ ﻣﯽداﻧﻢ‪ ،‬و آن اﯾﻨﮑ...,‫ﯾادتو آنچنان دﯾوانهام کرده است که به اندﯾشه‌های خوﯾش اطمﯿنان ندارم‪. ‬ﯾک چﯿز مﯽدانم‪، ‬و آن اﯾن...,‫یﺎدتو آذچنﺎن دیواذهام اذدیشههﺎی اطمینﺎن ذدارم‪ ‬یک میداذم‪ ‬و بﺎ همﮥ‬,یادتو آنچنان دیوانه‌ام کرده است که به اندیشه‌های خویش اطمینان ندارم . یک چیز می‌دانم‌ و آن اینکه...
1,"CARTWRIGHT, JULIA, Baldassare Cas","CARTWRIGHT, JULIA, Baldassare Cas",CARTWRIGHT JULIA Baldassare Cas,CARTWRIGHT JULIA Baldassare Cas
2,‫ﻧﻤﯿﮕﻮﯾﺪ‪ .‬اﻟﺒﺘﻪ ﻫﺰﯾﻮد‪ ،‬در ﭘﺎﯾﺎن ﻣﻄﺎﻟﺒﯽ ﮐﻪ از ﺗﺌﻮﮔﻮﻧﯿﺎ ﺑﺮ ﺟﺎي ﻣﺎﻧﺪه اﺳﺖ‪ ،‬از روزﮔﺎراﻧﯽ ﮐﻪ زﻧ...,‫نمﯿگوﯾد‪. ‬البته هزﯾود‪، ‬در پاﯾان مطالبﯽ که از تیوگونﯿا بر جای مانده است‪، ‬از روزگارانﯽ که زن...,‫ذمیگوید‪ ‬اﻟپته هزیود‪ ‬در پﺎیﺎن مطﺎﻟپی تﺌوگوذیﺎ جﺎی مﺎذده است‪ ‬از روزگﺎراذی زذﺎن قهرمﺎن مردان‬,نمیگوید . البته هزیود در پایان مطالبی که از تئوگونیا بر جای مانده است‌ از روزگارانی که زنان قهرم...
3,‫اﺻﻼح ﺧﻂ و ﺗﻨﻈﯿﻢ اﻟﻔﺒﺎي ﭼﯿﻨﯽ ﮐﻨﻮﻧﯽ ﺑﻮد‪ ،‬ﺑﺎ ﺳﺨﻨﯽ ﺗﺎرﯾﺨﯽ‪ ،‬ﮐﻪ ﺧﻮﺷﺎﯾﻨﺪ ادﯾﺒﺎن ﭼﯿﻨﯽ ﻧﺒﻮد‪ ،‬ﺑﻪ اﯾ...,‫اصلاح خط و تنظﯿم الفبای چﯿنﯽ کنونﯽ بود‪، ‬با سخنﯽ تارﯾخﯽ‪، ‬که خوشاﯾند ادﯾبان چﯿنﯽ نبود‪، ‬به ا...,‫اصﻼح خط تنظیم اﻟفپﺎی چینی کنوذی بود‪ ‬بﺎ سخنی تﺎریخی‪ ‬که خوشﺎیند ادیپﺎن چینی ذپود‪ ‬به اذتﻘﺎدات‬,اصلاح خط و تنظیم الفبای چینی کنونی بود با سخنی تاریخی‌ که خوشایند ادیبان چینی نبود به این انتقادات
4,‫ﺑﺰرگ ﯾﻮﻧﺎن و روم‪ ،‬و از ﻟﺤﺎظ ﻓﺼﺎﺣﺖ ﺑﯿﺎن و ﻇﺮاﻓﺖ ﺗﺸﺒﯿﻬﺎت ادﺑﯽ ﺑﺎ ﻏﺰﻟﺴﺮاﯾﺎن اﯾﺘﺎﻟﯿﺎﯾﯽ رﻗﺎﺑﺖ ﮐﻨﻨﺪ...,‫بزرگ ﯾونان و روم‪، ‬و از لحاظ فصاحت بﯿان و ظرافت تشبﯿهات ادبﯽ با غزلسراﯾان اﯾتالﯿاﯾﯽ رقابت کنند...,‫بزرگ یوذﺎن روم‪ ‬و ﻟحﺎظ فصﺎحت بیﺎن ظرافت تشپیهﺎت ادبی بﺎ غزﻟسرایﺎن ایتﺎﻟیﺎیی رقﺎبت کنند‪ ‬‬,بزرگ یونان و روم و از لحاظ فصاحت بیان و ظرافت تشبیهات ادبی با غزلسرایان ایتالیایی رقابت کنند .
5,‫ﻣﺘﺤﺪه را رﻫﺒﺮي ﻣﯿﮑﺮد‪ .‬وي ﻫﻨﻮز ﺧﺎﻃﺮه زﻣﺎﻧﯽ را ﮐﻪ در ﻓﺮاﻧﺴﻪ ﺑﻪ ﺳﻤﺖ ﻓﺮﺳﺘﺎده ﻫﻠﻨﺪ اﻧﺠﺎم وﻇﯿﻔﻪ ﻣﯿﮑ...,‫متحده را رهبری مﯿکرد‪. ‬وی هنوز خاطره زمانﯽ را که در فرانسه به سمت فرستاده هلند انجام وظﯿفه مﯿک...,‫متحده رهپری میکرد‪ ‬وی خﺎطره زمﺎذی فراذسه فرستﺎده هلند اذجﺎم وظیفه میکرد ﻟوووا‬,متحده را رهبری میکرد . وی هنوز خاطره زمانی را که در فرانسه به سمت فرستاده هلند انجام وظیفه میکرد...
6,‫ﻣﯽﭘﺮداﺧﺖ‪ .‬ﻣﻨﻈﻮﻣﻪاي ﺑﻪ ﻧﺎم »ﭘﯿﺮوزي ﺣﯿﺎت« ﮐﻪ ﭘﺲ از ﺳﺮودن ‪ 544‬ﻣﺼﺮع آن‪ ،‬در ﻧﺘﯿﺠﮥ ﺳﻔﺮ آﺧﺮﯾﻦ ﺷﺎ...,‫مﯽپرداخت‪. ‬منظومهای به نام» پﯿروزی حﯿات «که پس از سرودن ‪ ۵۴۴ ‬مصرع آن‪، ‬در نتﯿجه سفر آخرﯾن ش...,‫میپرداخت‪ ‬منظومهای ذﺎم» پیروزی حیﺎت «که سرودن ‪ ‬مصرع آن‪ ‬در ذتیجﮥ سفر آخرین شﺎعر‪ ‬‬,می‌پرداخت‌ . منظومه‌ای به نام پیروزی حیات که پس از سرودن 544مصرع آن در نتیجه سفر آخرین شاعر
7,‫ﮐﻪ دارﻧﺪ ﺑﻪ وﺻﻒ در آﻣﺪﻫﺎﻧﺪ; ﺟﻤﻠﮕﯽ ﺑﺎ ﭼﻨﺎن اﺷﻌﺎر ﻣﻮزون و ﺑﻠﻨﺪﭘﺎﯾﻬﺎي ﮐﻪ ﮐﺸﻮر ﻓﺮاﻧﺴﻪ ﭘﺲ از روﻧﺴﺎر ...,‫که دارند به وصف در آمدهاند; جملگﯽ با چنان اشعار موزون و بلندپاﯾهای که کشور فرانسه پس از رونسار ...,‫که دارذد وصف آمدهﺎذد جملگی بﺎ چنﺎن اشﻌﺎر موزون بلندپﺎیهﺎی کشور فراذسه روذسﺎر مﺎذند را‬,که دارند به وصف در آمدهاند جملگی با چنان اشعار موزون و بلندپایهای که کشور فرانسه پس از رونسار ما...
8,‫ﺷﻮد و اﯾﻦ راي ﺧﻮد را ﺑﻪ ﻣﻨﺎﺳﺒﺖ ﻧﺎزاﯾﯽ ﻣﻠﮑﻪ ﺑﻪ ﭼﺎرﻟﺰ دوم ﭘﯿﺸﻨﻬﺎد ﮐﺮد; اﻣﺎ ﭼﺎرﻟﺰ از ﺧﻮار ﮐﺮدن ﺑﯿﺸ...,‫شود و اﯾن رای خود را به مناسبت نازاﯾﯽ ملکه به چارلز دوم پﯿشنهاد کرد; اما چارلز از خوار کردن بﯿش...,‫شود رای منﺎسپت ذﺎزایی ملکه چﺎرﻟز پیشنهﺎد امﺎ چﺎرﻟز خوار زن خود‬,شود و این رای خود را به مناسبت نازایی ملکه به چارلز دوم پیشنهاد کرد اما چارلز از خوار کردن بیشتر...
9,‫‪ 500‬ﻣﺘﺮ ﻃﻮل دارد؛ و ﭼﻨﺪﯾﻦ ﺣﯿﺎط در آن ﺳﺎﺧﺘﻪ ﺷﺪه‪ ،‬و زﻣﺎﻧﯽ ‪ 86000‬ﻣﺠﺴﻤﻪ در آن ﺟﺎي داﺷﺘﻪ اﺳﺖ؛ ...,‫‪ ۵۰۰ ‬متر طول دارد؛ و چندﯾن حﯿاط در آن ساخته شده‪، ‬و زمانﯽ ‪ ۸۶۰۰۰ ‬مجسمه در آن جای داشته است...,‫‪ ‬متر طول دارد؛ حیﺎط سﺎخته شده‪ ‬و زمﺎذی ‪ ‬مجسمه جﺎی است؛ مجموعﮥ‬,500متر طول دارد و چندین حیاط در آن ساخته شده و زمانی 86000مجسمه در آن جای داشته است مجموعه‌


## Sentence Tokenization

<h5 style='direction:rtl;'> در این بخش ابتدا متن هر کتاب جدا گانه نرمالایز می شود: </h5>

In [13]:
# normalizing the divided text
Books_Dict_Normalized = {}

for vol in tqdm.tqdm(Books_Dict_Line):
    Books_Dict_Normalized[vol] = [alireza_normalizer.normalize(line) for line in Books_Dict_Line[vol]]

100%|██████████| 11/11 [00:12<00:00,  1.11s/it]


<h5 style='direction:rtl;'> سپس در این قسمت تمام خطوط هر کتاب در ادامه هم تبدیل به یک تک رشته می شوند و سپس توسط یک توکنایزر جمله جملات از هم جدا می شوند. دلیل اینکه ابتدا خطوط بهم می چسبند این است که بعضی جملات در چند خط آماده اند. </h5>

In [14]:
# adding a space to beginning of each line and concating each book in a single string
Books_Dict_S = {}
sentence_tokenizer = hazm.SentenceTokenizer()

for vol in tqdm.tqdm(Books_Dict_Normalized):
    full_txt = ''
    for line in Books_Dict_Normalized[vol]:
        full_txt += " " + line
    Books_Dict_S[vol] = sentence_tokenizer.tokenize(full_txt)

# removing extra spaces
Books_Dict_Sent = {}
for vol in tqdm.tqdm(Books_Dict_Line):
    Books_Dict_Sent[vol] = [alireza_normalizer.normalize(sent) for sent in Books_Dict_S[vol]]

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:10<00:00,  1.07it/s]


<h5 style='direction:rtl;'> در این بخش نسبت جملات به تعداد خطوط در هر کتاب مورد بررسی قرار گرفته است: </h5>

In [15]:
df = pd.DataFrame({'Book Names':Books_Dict_Sent.keys(),
                    'Sentences Count':[len(Books_Dict_Sent[i]) for i in Books_Dict_Sent],
                    'Lines Count':[len(Books_Dict_Line[i]) for i in Books_Dict_Line],
                    'Sentences per Lines Ratio':[len(Books_Dict_Sent[i])/len(Books_Dict_Line[i]) for i in Books_Dict_Sent]})

display(df)

,Book Names,Sentences Count,Lines Count,Sentences per Lines Ratio
0,مشرق زمین، گهواره تمدن,13521,22143,0.610622
1,یونان باستان,12889,16448,0.783621
2,سزار و مسیح,13604,17912,0.759491
3,عصر ایمان,23059,33810,0.682017
4,رنسانس,12006,18034,0.665742
5,اصلاح دین,20196,26727,0.755640
6,آغاز عصر خرد,13568,17433,0.778294
7,عصر لویی چهاردهم,15077,19121,0.788505
8,عصر ولتر,16799,21010,0.799572
9,روسو و انقلاب,24519,31057,0.789484


## Word Tokenization

<h5 style='direction:rtl;'>  در این قسمت ابتدا هر جمله به کلمات تجزیه می شود و سپس کلمات پرتکرار نمایش داده می شود: </h5>

In [16]:
word_tokenizer = hazm.WordTokenizer()
text_tokens = [[word_tokenizer.tokenize(sent) for sent in Books_Dict_Sent [sents]] for sents in tqdm.tqdm(Books_Dict_Sent)]

100%|██████████| 11/11 [00:04<00:00,  2.72it/s]


## Frequency Analysis

In [36]:
Books_Tokens_Dict = {}

for vol in tqdm.tqdm(Books_Dict_Sent):
    tokens_list = []

    for sent in Books_Dict_Sent[vol]:
        tokens_list.extend(word_tokenizer.tokenize(sent))
    Books_Tokens_Dict[vol] = tokens_list

All_Tokens_List = []
for vol in tqdm.tqdm(Books_Dict_Sent):
    All_Tokens_List.extend(Books_Tokens_Dict[vol])

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:00<00:00, 233.97it/s]


<h5 style='direction:rtl;'> در این قسمت تعداد کل کلمات، تعداد کلمات متمایز، متوسط حرف در هر کلمه و طولانی ترین کلمه برای هر کتاب به نمایش در آمده است. </h5>

In [31]:
df = pd.DataFrame({'Book Names':Books_Tokens_Dict.keys(),
                    'Number of words':[len(Books_Tokens_Dict[i]) for i in Books_Tokens_Dict],
                    'Number of unique words':[len(set(Books_Tokens_Dict[i])) for i in Books_Tokens_Dict],
                    'Average word length':[np.sum([len(word) for word in Books_Tokens_Dict[i]])/len(Books_Tokens_Dict[i]) for i in Books_Tokens_Dict],
                    'Longest word':[Books_Tokens_Dict[i][np.argmax([len(word) for word in Books_Tokens_Dict[i]])] for i in Books_Tokens_Dict]})
display(df)


,Book Names,Number of words,Number of unique words,Average word length,Longest word
0,مشرق زمین، گهواره تمدن,417019,30193,3.891962,مونتسستوارتالفینستن
1,یونان باستان,322726,23392,3.806867,chermatistikeواژههای
2,سزار و مسیح,348050,26258,3.914880,واگذاشته_شده_بودند
3,عصر ایمان,668583,36590,3.838898,homoousiosمی‌دانستند
4,رنسانس,345111,27458,4.019779,فرانچسکوتورنابوئونی
5,اصلاح دین,534559,33032,3.832320,وآدابوآیینهایمشرکانهدر
6,آغاز عصر خرد,349523,22909,3.797687,استرایمبردوبوالامبر
7,عصر لویی چهاردهم,387351,26988,3.781715,پراکنده_شده_بودند
8,عصر ولتر,389600,29194,4.028978,بریفه‌دینویسته‌لیتراتوربترفند
9,روسو و انقلاب,611569,32534,3.832694,یوهانفریدریش‌تیشباین


### Word Frequancy (with stop words)

<h5 style='direction:rtl;'> در این بخش به کمک ابزار FreqDist کتابخانه nltk کلمات پرتکرار مشخص شدند. </h5>

In [18]:
from nltk import FreqDist

dataframe = {}
for vol in Books_Tokens_Dict:
     dataframe[vol] = FreqDist(Books_Tokens_Dict[vol]).most_common(10)

freq_analysis = pd.DataFrame(dataframe)
display(freq_analysis)

,مشرق زمین، گهواره تمدن,یونان باستان,سزار و مسیح,عصر ایمان,رنسانس,اصلاح دین,آغاز عصر خرد,عصر لویی چهاردهم,عصر ولتر,روسو و انقلاب,عصر ناپلئون
0,"(و, 24573)","(و, 18445)","(و, 18044)","(و, 32971)","(و, 15679)","(و, 28661)","(و, 17112)","(و, 17280)","(., 18233)","(و, 25665)","(و, 23726)"
1,"(., 14576)","(., 12887)","(., 14200)","(به, 24261)","(., 12522)","(., 20195)","(., 13568)","(., 15077)","(و, 18010)","(., 25468)","(., 18650)"
2,"(به, 13619)","(به, 10312)","(به, 11991)","(., 23445)","(به, 11637)","(به, 18551)","(به, 12691)","(به, 13425)","(را, 12656)","(به, 21648)","(به, 17994)"
3,"(که, 12923)","(در, 9566)","(را, 10900)","(در, 19715)","(در, 10508)","(در, 15663)","(در, 11712)","(در, 11947)","(به, 12586)","(در, 19141)","(در, 16377)"
4,"(از, 12123)","(که, 8983)","(در, 10232)","(از, 19071)","(را, 10264)","(را, 15514)","(که, 10113)","(را, 11075)","(در, 11446)","(که, 17131)","(که, 13992)"
5,"(در, 11963)","(را, 8813)","(از, 9524)","(که, 17807)","(از, 9360)","(از, 14276)","(را, 9589)","(که, 10711)","(از, 11394)","(از, 15769)","(از, 13070)"
6,"(را, 10844)","(از, 8746)","(که, 9467)","(را, 17559)","(که, 9252)","(که, 13486)","(از, 9563)","(از, 9712)","(که, 10484)","(را, 15724)","(را, 12681)"
7,"(این, 5413)","(), 3768)","(با, 3741)","(این, 7304)","(با, 4197)","(), 7237)","(), 4901)","(), 6567)","(با, 5085)","(او, 7982)","(با, 5253)"
8,"(آن, 5211)","((, 3766)","(این, 3342)","(بود, 6194)","(او, 4185)","((, 7219)","((, 4880)","((, 6546)","(این, 4298)","(این, 7981)","(آن, 5020)"
9,"(است, 4169)","(این, 3409)","(آن, 3287)","(با, 5880)","(آن, 3823)","(با, 6013)","(او, 3971)","(با, 4475)","(او, 3028)","(با, 6920)","(این, 4849)"


<h5 style='direction:rtl;'> همانطور که پیشبینی می شد کلمات با فرکانس بالا، کلمات دارای اطلاعات نبودند بلکه حروف ربط و فعل های پرتکرار بودند. برای رفع این چالش در بخش بعدی stopword ها از کلمات حذف می شوند. </h5>

### Word Frequancy (without stop words)

In [20]:
file1 = open('NLP_Homeworks/HW1/persian_stopwords.txt')
Lines = file1.readlines()
file1.close()

stopwords = []
for line in Lines:
  line = line.strip()
  stopwords.append(line)

stopwords += ['-','،','؛','»','.',')','(','\u202a','\u202c\u202c','\u202a']

<h5 style='direction:rtl;'> اکنون که stopword ها حذف شدند کلمات پر تکرار هر کتاب معنی دار و منطقی شده است. مثلا برای جلد رنسانس کلمات رم، ایتالیا فلورانس، ونیز و پاپ از کلمات پر تکرار اند یا برای عصر ناپلئون کلمات ناپلئون، فرانسه و انقلاب کلمات پرتکرارند. چون متن کتاب تاریخی است بعضی کلمات پرتکرار اطلاعات ارزشمندی ندارند و باید جزو stopword ها حساب شوند، مانند شهر، سال، مردم، دست و... . </h5>

In [21]:
Books_Tokens_Dict_nonstop  = {vol:[t for t in Books_Tokens_Dict[vol] if t not in stopwords] for vol in tqdm.tqdm(Books_Tokens_Dict)}

dataframe_nonstop = {}
for vol in Books_Tokens_Dict_nonstop:
     dataframe_nonstop[vol] = FreqDist(Books_Tokens_Dict_nonstop[vol]).most_common(10)

freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop)

display(freq_analysis_nonstop)

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:11<00:00,  1.05s/it]


,مشرق زمین، گهواره تمدن,یونان باستان,سزار و مسیح,عصر ایمان,رنسانس,اصلاح دین,آغاز عصر خرد,عصر لویی چهاردهم,عصر ولتر,روسو و انقلاب,عصر ناپلئون
0,"(مردم, 1180)","(یونان, 984)","(سال, 1185)","(سال, 1742)","(پاپ, 989)","(سال, 1873)","(سال, 1343)","(سال, 1209)","(مردم, 1323)","(سال, 1864)","(ناپلئون, 2005)"
1,"(دست, 1096)","(آتن, 877)","(روم, 1086)","(دست, 1287)","(سال, 928)","(دست, 1386)","(دست, 681)","(فرانسه, 867)","(ولتر, 1265)","(عنوان, 1359)","(فرانسه, 1475)"
2,"(کار, 992)","(مردم, 844)","(دست, 675)","(قرن, 1221)","(رم, 755)","(مردم, 1062)","(صورت, 609)","(دست, 803)","(فرانسه, 1042)","(قرار, 1316)","(سال, 1278)"
3,"(سال, 732)","(دست, 750)","(کار, 625)","(مردم, 1102)","(ایتالیا, 738)","(لوتر, 942)","(پادشاه, 598)","(انگلستان, 691)","(دست, 822)","(دست, 1215)","(دست, 1001)"
4,"(صورت, 705)","(میکند, 657)","(رم, 588)","(کار, 1035)","(کار, 716)","(شهر, 906)","(کار, 573)","(میکرد, 648)","(انگلستان, 740)","(فرانسه, 1112)","(عنوان, 951)"
5,"(چین, 686)","(سال, 634)","(مردم, 577)","(کلیسا, 986)","(فلورانس, 714)","(کلیسا, 901)","(انگلستان, 568)","(لویی, 552)","(کتاب, 677)","(کار, 1068)","(کار, 801)"
6,"(هند, 669)","(شهر, 619)","(رومی, 553)","(شهر, 935)","(دست, 705)","(پاپ, 898)","(عنوان, 567)","(کار, 541)","(ساخت, 639)","(زندگی, 935)","(قرار, 770)"
7,"(زمان, 609)","(یونانی, 606)","(شهر, 533)","(سر, 875)","(شهر, 672)","(کتاب, 855)","(مردم, 560)","(قرار, 517)","(انسان, 634)","(میکرد, 933)","(صورت, 659)"
8,"(زن, 556)","(کار, 606)","(قیصر, 515)","(لکن, 862)","(ونیز, 574)","(فرانسه, 848)","(فرانسه, 558)","(زندگی, 478)","(شاه, 585)","(روسو, 879)","(زمان, 608)"
9,"(زندگی, 545)","(قرار, 511)","(سر, 502)","(کتاب, 853)","(زندگی, 531)","(انگلستان, 837)","(میکرد, 552)","(کتاب, 477)","(جهان, 563)","(نام, 874)","(انقلاب, 576)"


## Keyword Extraction

<h5 style='direction:rtl;'> در این بخش به کمک کتابخانه yake عبارات کلیدی استخراج شده اند که نتایج آن بسیار دقیق تر و با معناتر از بخش قبل است. به عنوان مثال برای یونان باستان عبارات: جنگ آتن و سپارت، عصر پریکلس، یونان قرن پنجم و ... عبارات کلیدی اند. در این بخش ابتدا توکن های بدون stopword به هم ملحق شده اند سپس عبارات کلیدی استخراج شده است. بقیه کد برای نمایش بهتر نتایج است از جمله رند کردن احتمال تکرار عبارت، نمایش بهتر و چند ایندکسی از دیتا فریم و مرتب کردن از احتمال زیاد به کم سطرها. </h5>

In [22]:
import yake

# building the text from nonstop tokens
Books_Sent_Dict_nonstop = {}
for vol in tqdm.tqdm(Books_Tokens_Dict_nonstop):
    Books_Sent_Dict_nonstop[vol] = ' '.join(Books_Tokens_Dict_nonstop[vol])

kw_extractor = yake.KeywordExtractor()
Books_KW_List = []

for vol in tqdm.tqdm(Books_Tokens_Dict_nonstop):
    arr = np.array(kw_extractor.extract_keywords(Books_Sent_Dict_nonstop[vol]))
    rate = np.array(arr[:,1], dtype=float).round(decimals=8) #to make the table looks better the numbers are rounded
    Books_KW_List.append(np.array([arr[:,0],rate]).transpose())

# making multi-index
col1 = []
for i in list(Books_Tokens_Dict_nonstop.keys()):
    col1.append(i)
    col1.append(i)

col2 = []
for i in range(len(Books_Tokens_Dict_nonstop)):
    col2.append('Keyword')
    col2.append('Rate')
cols = [np.array(col1), np.array(col2)]

# reshaping the keyword dictionart so it works fine with multi-index
final_array = np.flip(np.array(Books_KW_List).transpose((1, 0, 2)).reshape(20,22),0)
freq_analysis_nonstop = pd.DataFrame(final_array, columns=cols)
pd.set_option('display.max_columns', None)
display(freq_analysis_nonstop)

100%|██████████| 11/11 [02:31<00:00, 13.79s/it]


مشرق زمین، گهواره تمدن                    یونان باستان             \
                  Keyword       Rate              Keyword       Rate   
0         انسان نام انسان  2.274e-05       پایان قرن پنجم   8.04e-05   
1        آیین بودایی ژاپن  2.248e-05     آتن شهرهای یونان  8.017e-05   
2           هزار سال آیین  2.163e-05   شهرهای یونانی آسیا  7.758e-05   
3           چین ژاپن صورت  2.106e-05           عصر پریکلس  7.553e-05   
4      آیین کنفوسیوس آیین  2.071e-05                  دست  7.369e-05   
5          هزار سال میلاد  1.951e-05             قرن پنجم   7.29e-05   
6            هزار سال هند  1.942e-05         شهرهای یونان  6.909e-05   
7         تقریبا هزار سال  1.927e-05  شهرهای یونان اسپارت  6.702e-05   
8        انسان عصر نوسنگی  1.917e-05       جنگ آتن اسپارت  6.536e-05   
9           مردم ژاپن عصر  1.906e-05                 مردم  6.443e-05   
10      انسان اولیه انسان  1.698e-05       یونان قرن پنجم  6.132e-05   
11         سال میلاد آثار  1.611e-05                  آتن  5.981e-05   
12         عصر نوسنگی دست  1.535e-05           مردم یونان  5.582e-05   
13        عصر دیرینه سنگی  1.516e-05                یونان  4.994e-05   
14               هزار سال  1.499e-05       آتن عصر پریکلس  3.884e-05   
15          آیین بودا هند   1.39e-05      پریکلس مردم آتن  3.859e-05   
16          هزار سال زمان  1.325e-05             مردم آتن  3.667e-05   
17         تاریخ هزار سال  1.293e-05    کشور شهرهای یونان  3.655e-05   
18        دوره سلطنت قدیم   8.93e-06     یونان عصر پریکلس   2.91e-05   
19         آیین بودا آیین   8.85e-06      شبه جزیره یونان  2.198e-05   

              سزار و مسیح                       عصر ایمان             \
                  Keyword       Rate              Keyword       Rate   
0       امپراطوری روم حفظ  2.964e-05    کلیسای جامع شارتر  1.358e-05   
1            دست هزار سال  2.831e-05            کتاب مقدس  1.308e-05   
2         هزار پیاده هزار  2.829e-05          قرون وسطایی  1.275e-05   
3            مسیح پسر خدا  2.784e-05          قرن دوازدهم  1.233e-05   
4                     روم  2.759e-05    انسان قرون وسطایی  1.211e-05   
5         فلسفه یونان روم   2.74e-05       قرن دهم میلادی  1.203e-05   
6       نامزد مقام کنسولی  2.727e-05     مسیحی قرن سیزدهم  1.102e-05   
7   هنگامی مارکوس آورلیوس  2.683e-05     اروپای قرون وسطی  1.038e-05   
8          روم بازگشت سال  2.673e-05    امپراطور روم شرقی   9.98e-06   
9                     سال  2.592e-05      کلیسای جامع شهر    9.5e-06   
10           مردم روز روز   2.48e-05          کلیسای جامع   8.75e-06   
11      یونانی قرن میلادی  2.452e-05      مسیحی قرون وسطی   8.45e-06   
12        تاریخ روم تاریخ  2.444e-05   امپراطوری مقدس روم   7.85e-06   
13         قیصر بیست هزار  2.429e-05     پایان قرن سیزدهم   7.63e-06   
14        زبان یونانی کار  2.391e-05    یهودیان قرون وسطی   6.64e-06   
15    زمان مارکوس آورلیوس  2.255e-05           قرن سیزدهم   6.47e-06   
16        قیصر هنگامی سال  2.082e-05    کتاب مقدس یهودیان   5.63e-06   
17           هزار مرد کار   1.99e-05      تاریخ قرون وسطی   5.35e-06   
18      مردم روم آوگوستوس  1.983e-05            قرون وسطی   4.85e-06   
19          روم زمان نرون  1.172e-05  قرون دوازدهم سیزدهم   4.01e-06   

                  رنسانس                        اصلاح دین            \
                 Keyword      Rate                Keyword      Rate   
0     خاندان مدیچی حکومت  1.43e-06          کتاب مقدس پاپ  4.73e-06   
1     کلیسای سانتا ماریا  1.38e-06       لاتینی کتاب مقدس   4.7e-06   
2         حضرت مریم کودک  1.34e-06           چشم جهان فرو  4.54e-06   
3    کشور شهرهای ایتالیا  1.26e-06         دینی کتاب مقدس  4.43e-06   
4   فلورانس خاندان مدیچی  1.19e-06  آیین کاتولیک انگلستان  4.12e-06   
5         مریم عذرا کودک   1.1e-06         لوتر کتاب مقدس  3.73e-06   
6     ساختمان کلیسای سان  1.08e-06        دوره اصلاح دینی  3.55e-06   
7       کلیسای سان مارکو  1.03e-06             اصلاح دینی  3.42e-06   
8       مریم کلیسای قدیس   8.5e-07      شورای عمومی کلیسا  3.29e-06   
9        تصویر مریم عذرا   8.3e-07    تفتیش افکار اس